In [24]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import getpass
import time
import random
import requests

To move forward with the project, you need to create a collection of songs with their audio features - as large as possible!

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster. The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

Goal:

song ID | danceability | energy |...|
---|---|---|---|
value|value|value|value|

generate dataframe for one song, one playlist, multiple playlist



In [ ]:
client_id = getpass.getpass(prompt="Spotify client_id: ")
client_secret = getpass.getpass(prompt="Spotify client_secret: ")

#Initialize a SpotiPy instance with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret)
                    )

Get list of categories 

In [3]:
category_id_1= []
for cat in sp.categories()['categories']['items']:
    category_id_1.append(cat['id'])

In [5]:
#assigned list
category_id =['wellness',
 'focus',
 'sleep',
 'party',
 'indie_alt',
 'dinner',
 'jazz',
 'mood',
'decades',
'pop',
'rock']

get lists of playlist

In [6]:
list_of_playlists_1 = []
for item in category_id:
    for j in range(len(sp.category_playlists(item)['playlists']['items'])):
        try:
            list_of_playlists_1.append(sp.category_playlists(item)['playlists']['items'][j]['id'])
        except:
            list_of_playlists_1.append(None)
            
            

In [7]:
list_of_playlists = [x for x in list_of_playlists_1 if x is not None]

In [8]:
list_of_playlists

['37i9dQZF1DX9uKNf5jGX6m',
 '37i9dQZF1DXdejmG21jbny',
 '37i9dQZF1DWZqd5JICZI0u',
 '37i9dQZF1DWXe9gFZP0gtP',
 '37i9dQZF1DXcBbGCLlic3p',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DWZeKCadgRdKQ',
 '37i9dQZF1DX9sIqqvKsjG8',
 '37i9dQZF1DX7K31D69s4M1',
 '37i9dQZF1DXa2SPUyWl8Y5',
 '37i9dQZF1DWSw8liJZcPOI',
 '37i9dQZF1DWWn6teJIIcfG',
 '37i9dQZF1DWXUpC6mczRpA',
 '37i9dQZF1DX4FLflF8wPKE',
 '37i9dQZF1DX3DZBe6wPMXo',
 '37i9dQZF1DX7KrTMVQnM02',
 '37i9dQZF1DWZd79rJ6a7lp',
 '37i9dQZF1DX9if5QDLdzCa',
 '37i9dQZF1DXbZmKskFbVct',
 '37i9dQZF1DXdbkmlag2h7b',
 '37i9dQZF1DXaXB8fQg7xif',
 '37i9dQZF1DX7F6T2n2fegs',
 '37i9dQZF1DX0Uv9tZ47pWo',
 '37i9dQZF1DX8FwnYE6PRvL',
 '37i9dQZF1DX6GwdWRQMQpq',
 '37i9dQZF1DX8a1tdzq5tbM',
 '37i9dQZF1DWUq3wF0JVtEy',
 '37i9dQZF1DXa8n42306eJB',
 '37i9dQZF1DX873GaRGUmPl',
 '37i9dQZF1DXaLI8a395lse',
 '37i9dQZF1DXdTCdwCKzXwo',
 '37i9dQZF1DX0YKekzl0blG',
 '37i9dQZF1DWTSKFpOdYF1r',
 '37i9dQZF1DX8CopunbDxgW',
 '37i9dQZF1DXaVgr4Tx5kRF',
 '37i9dQZF1DXb3ZjVksUlfu',
 '37i9dQZF1DWSrj7tqQ9IOu',
 

In [9]:
#import urllib.request as urllib2
#from urllib2 import HTTPError
#list_of_playlists = []
#for i in range(len(category_id)):
   # try:
    #    list_of_playlists.append(sp.category_playlists(str(i))['playlists']['items']['id'])
 #   except HTTPError as err:
         #  if err.code == 404:
    #            print(sp.category_playlists(str(i))['playlists']['items']['id'], 'missing') 
         #  else:
          #     raise

Get tracks info from playlists

In [10]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks('spotify', playlist_id)
    tracks = results['items']
    while results['next']:
        # print(results['next'])
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [11]:
#def get_tracks_info(playlist_id):
songs_names = []
songs_ids = []
artist_name = []
features= []
def track_info(playlist_id):
    #features= []
    playlist_track = get_playlist_tracks("spotify", playlist_id)

    for i in range(len(playlist_track)):
            artists=[]
            for a in range(len(playlist_track[i]['track']['artists'])):
                    artists.append(playlist_track[i]['track']['artists'][a]['name'])
            artist_name.append(artists)
            songs_ids.append(playlist_track[i]['track']['id'])
            songs_names.append(playlist_track[i]['track']['name'])
            try:
                features.append(sp.audio_features(playlist_track[i]['track']['id']))
            except:
                features.append([{'danceability':None,
                                   'energy': None,
                                   'key': None,
                                   'loudness': None,
                                   'mode': None,
                                   'speechiness': None,
                                   'acousticness': None,
                                   'instrumentalness': None,
                                   'liveness': None,
                                   'valence': None,
                                   'tempo': None,
                                   'type': 'empty',
                                   'id': 'empty',
                                   'uri': 'empty',
                                   'track_href': 'empty',
                                   'analysis_url': 'empty',
                                   'duration_ms': None,
                                   'time_signature': None}])


In [12]:
#sp.playlist_items(playlist_id)["items"]

In [13]:
for i in list_of_playlists:
    track_info(i)

In [14]:
df = pd.DataFrame({"songs_names": songs_names, "songs_ids": songs_ids, "artists_names": artist_name})

In [15]:
df

,songs_names,songs_ids,artists_names
0,Astral Wonder,7En6m2Yg3J1zAq895bxl78,[Moon Oceans]
1,Evergreen,633YySlXq6qxSdCLFovXdv,[Cooper Sams]
2,Karlavagnen,7kdnp9XpskkNjGDRlmb8nw,[Karlavagnen]
3,Montana,0gBjWQIJ9KcdzT26UzkNqn,[Isobel Belano]
4,Electron,0KkNeFOoAKW0NkhaleFeGk,[Sisi Row]
...,...,...,...
10131,A Lull In Traffic,2HmdQtNJDraTM9JNQCXC0K,[The Gloria Record]
10132,Everyone Is My Friend,5BcarBozJi3Gv0ncP721y5,[OWLS]
10133,Earthquakes and Sharks,7jATtiR7Sjfke3qKdA5HEd,[Brandtson]
10134,A Life of Possibilities,3TKWCQMU7xYEEslFXsj51i,[Dismemberment Plan]


In [16]:
#for item in songs_ids:
 #   get_features(item)

In [17]:
feat_df = pd.DataFrame(features).stack().apply(pd.Series)

In [18]:
feat_df.reset_index()

,level_0,level_1,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,0,0.1710,0.1430,0,-24.178,1,0.0431,0.95200,0.92400,0.1110,0.0374,72.116,audio_features,7En6m2Yg3J1zAq895bxl78,spotify:track:7En6m2Yg3J1zAq895bxl78,https://api.spotify.com/v1/tracks/7En6m2Yg3J1z...,https://api.spotify.com/v1/audio-analysis/7En6...,181091,5
1,1,0,0.1310,0.0860,6,-20.439,1,0.0332,0.99300,0.87700,0.0794,0.0397,54.749,audio_features,633YySlXq6qxSdCLFovXdv,spotify:track:633YySlXq6qxSdCLFovXdv,https://api.spotify.com/v1/tracks/633YySlXq6qx...,https://api.spotify.com/v1/audio-analysis/633Y...,225882,5
2,2,0,0.0916,0.0343,11,-26.745,1,0.0419,0.84000,0.76200,0.1700,0.0424,82.815,audio_features,7kdnp9XpskkNjGDRlmb8nw,spotify:track:7kdnp9XpskkNjGDRlmb8nw,https://api.spotify.com/v1/tracks/7kdnp9XpskkN...,https://api.spotify.com/v1/audio-analysis/7kdn...,203455,5
3,3,0,0.3380,0.0901,3,-22.303,1,0.0323,0.76200,0.69900,0.0891,0.1150,135.633,audio_features,0gBjWQIJ9KcdzT26UzkNqn,spotify:track:0gBjWQIJ9KcdzT26UzkNqn,https://api.spotify.com/v1/tracks/0gBjWQIJ9Kcd...,https://api.spotify.com/v1/audio-analysis/0gBj...,166325,4
4,4,0,0.2500,0.2010,0,-21.141,1,0.0377,0.89500,0.96900,0.2270,0.0663,59.005,audio_features,0KkNeFOoAKW0NkhaleFeGk,spotify:track:0KkNeFOoAKW0NkhaleFeGk,https://api.spotify.com/v1/tracks/0KkNeFOoAKW0...,https://api.spotify.com/v1/audio-analysis/0KkN...,214800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10131,10131,0,0.3140,0.8380,4,-6.676,1,0.0336,0.00247,0.00136,0.1140,0.4280,146.008,audio_features,2HmdQtNJDraTM9JNQCXC0K,spotify:track:2HmdQtNJDraTM9JNQCXC0K,https://api.spotify.com/v1/tracks/2HmdQtNJDraT...,https://api.spotify.com/v1/audio-analysis/2Hmd...,245560,4
10132,10132,0,0.5300,0.7930,4,-7.662,1,0.0382,0.01320,0.00000,0.2360,0.6030,101.258,audio_features,5BcarBozJi3Gv0ncP721y5,spotify:track:5BcarBozJi3Gv0ncP721y5,https://api.spotify.com/v1/tracks/5BcarBozJi3G...,https://api.spotify.com/v1/audio-analysis/5Bca...,206351,4
10133,10133,0,0.6550,0.7030,1,-5.218,0,0.0296,0.01020,0.00000,0.4110,0.8930,112.047,audio_features,7jATtiR7Sjfke3qKdA5HEd,spotify:track:7jATtiR7Sjfke3qKdA5HEd,https://api.spotify.com/v1/tracks/7jATtiR7Sjfk...,https://api.spotify.com/v1/audio-analysis/7jAT...,177027,4
10134,10134,0,0.7090,0.8370,7,-7.273,1,0.0779,0.10100,0.51100,0.1140,0.5020,125.485,audio_features,3TKWCQMU7xYEEslFXsj51i,spotify:track:3TKWCQMU7xYEEslFXsj51i,https://api.spotify.com/v1/tracks/3TKWCQMU7xYE...,https://api.spotify.com/v1/audio-analysis/3TKW...,274720,4


In [19]:
final_df = pd.DataFrame(np.hstack([df, feat_df]), columns=df.columns.append(feat_df.columns))

In [20]:
final_df
final_df['artists_names'] = [','.join(map(str, l)) for l in final_df['artists_names']]

In [1]:
final_df = final_df.drop(['type', 'id', 'uri','track_href','analysis_url','duration_ms','time_signature'], axis=1)

NameError: name 'final_df' is not defined

Clustering

In [22]:
final_df.to_csv('ready_to_cluster.csv')